In [70]:
import scipy.io as sio
import pandas as pd

pacientes = ["P0" + str(i) + ".mat" for i in range(1,8)]
electrodes = [8, 44, 80, 131, 185]
sum_of_means_of_patients = dict.fromkeys(electrodes, [0 * 201])

for paciente in pacientes:
    for m in matrixes:
        test = sio.loadmat(paciente)["data"]
        for electrode in electrodes:
            dtf = pd.DataFrame(test[:,electrode,:])
            sum_of_means_of_patients[electrode] = map(sum, zip(sum_of_means_of_patients[electrode], dtf.mean(axis = 1).values))
             

for electrode in electrodes:
    means[electrode] = map(lambda a: a/len(pacientes), sum_of_means_of_patients[electrode])
    
print means
    
'''
a = pd.DataFrame({'data':range(10), 'data2': range(10,20)})
print a
print a.mean(axis=1)
    
print means
'''


{8: [6.2730851248187479e-06], 80: [2.3690875414068112e-06], 131: [-6.9009035229586333e-07], 44: [4.3670575506548145e-06], 185: [-5.7521734344352442e-06]}


"\na = pd.DataFrame({'data':range(10), 'data2': range(10,20)})\nprint a\nprint a.mean(axis=1)\n    \nprint means\n"